In [ ]:
import numpy as np
import robustsp as rsp

def arma_est_bip_s(x,p,q):
    
    if p==0 and q == 0:
        inno_scale = rsp.m_scale(x)
        print('at least p or q has to be non-zero')
    if len(x) <= p+q:
        print('There are too many parameters to estimate for chosen data size. Reduce model order or use a larger data set.')
        return
    
    # Robust starting point by BIP AR-S approximation
    beta_initial = rsp.robust_starting_point(x,p,q)
    
    F = lambda beta: rsp.arma_s_resid_sc(x, beta, p, q)
    
    F_bip = lambda beta: rsp.bip_s_resid_sc(x, beta, p, q)
    
    beta_arma = lsq(F, beta_hat_s,xtol=5*1e-5,ftol=5*1e-5,method='lm')[0]
    beta_bip = lsq(F_bip, beta_hat_s,xtol=5*1e-5,ftol=5*1e-5,method='lm')[0]
    
    a_sc = rsp.arma_s_resid_sc

In [7]:
import numpy as np
import robustsp as rsp
from robustsp.DependentData.Auxiliary.helper import poles as poles

def arma_s_resid_sc(x, beta_hat, p, q):
    
    phi_hat = beta_hat[p] if 0<p else []
    
    theta_hat = [] if q<0 else beta_hat[p:]
    
    N = len(x)
    r = max(p,q)
    a = np.zeros(N)
    x_sc = rsp.m_scale(x)
    
    if r==0:
        a_sc = np.array(x_sc)
        a = np.array(x)
    else:
        if poles(phi_hat) or poles(theta_hat):
            a_sc = 10**10
        elif p>=1 and q>=1:
            # ARMA Models
            for ii in range(r,N):
                # ARMA residuals
                a[ii] = x[ii]-phi_hat@x[ii-1:ii-p:-1]+\
                theta_hat@a[ii-1:ii-q:-1]
        elif p==0 and q>=1:
            # MA model
            for ii in range(r,N):
                a[ii]=x[ii]+theta_hat@a[ii-1:ii-q:-1]
        elif p>=1 and q==0:
            # AR model
            a[ii] = x[ii] - phi_hat@x[ii-1:ii-p:-1]
    
    a_sc = rsp.m_scale(a[p:])
    
    return a_sc

x = np.array([.4153,-.6244,-1.8424,-1.655])
b = np.array([1,2,3])
p=1
q=2
arma_s_resid_sc(x,b,p,q)

TypeError: 'numpy.int32' object is not iterable